In [2]:
# standard
import pandas as pd
import numpy as np
import random
import os

# tf and keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import models
from keras import layers
from tensorflow.keras.layers import GlobalAveragePooling2D
from PIL import ImageFile

# sklearn
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# plots
import seaborn as sns
import matplotlib.pyplot as plt

from PIL import Image, ImageOps

In [6]:
def load_metadata(metadata_path='fungi-clef-2025/metadata/FungiTastic-FewShot/', image_path='fungi-clef-2025/images/FungiTastic-FewShot/'):
    """Load the metadata for each data split."""
    # Load the metadata for each split
    train_metadata = pd.read_csv(os.path.join(metadata_path, 'FungiTastic-FewShot-Train.csv'))
    val_metadata = pd.read_csv(os.path.join(metadata_path, 'FungiTastic-FewShot-Val.csv'))
    test_metadata = pd.read_csv(os.path.join(metadata_path, 'FungiTastic-FewShot-Test.csv'))
    
    # Label each split
    train_metadata["split"] = "train"
    val_metadata["split"] = "val"
    test_metadata["split"] = "test"

    # Join all of the data together
    df_metadata = pd.concat([train_metadata, val_metadata, test_metadata])

    # Add the full image location for each image
    # Options for image size include 300p, 500p, 720p, fullsize 
    df_metadata["image_path"] = df_metadata.apply(
        lambda row: os.path.join(image_path, f"{row['split']}/300p/{row['filename']}"), axis=1
    )

    return df_metadata


def filter_low_counts(df, min_samples):
    """Filter out examples of fungi with low value counts."""
    class_counts = df["class"].value_counts()
    frequent_classes = class_counts[class_counts >= min_samples].index
    filtered_df = df[df["class"].isin(frequent_classes)]
    return filtered_df


def resize_with_aspect_ratio(image, target_size):
    # Get original dimensions
    width, height = image.size
    
    # Calculate scaling factor
    if width > height:
        new_width = target_size
        new_height = int(target_size * height / width)
    else:
        new_height = target_size
        new_width = int(target_size * width / height)
    
    # Resize the image
    return image.resize((new_width, new_height), Image.Resampling.LANCZOS)


def add_padding(image, target_size):
    # Calculate padding
    width, height = image.size
    delta_w = target_size - width
    delta_h = target_size - height
    padding = (delta_w // 2, delta_h // 2, delta_w - delta_w // 2, delta_h - delta_h // 2)
    
    # Add padding (black by default)
    return ImageOps.expand(image, padding, fill=(0, 0, 0))  # Use fill=(255,255,255) for white padding


def preprocess_image(image_path, target_size):
    # Open the image
    image = Image.open(image_path)
    
    # Resize while maintaining aspect ratio
    resized_image = resize_with_aspect_ratio(image, target_size)
    
    # Add padding to make it square
    padded_image = add_padding(resized_image, target_size)
    
    return padded_image


def load_images_and_labels(df, image_size):
    """Load the images and labels based on the metadata frame passed in."""
    images = []
    labels_class = []
    labels_poison = []
    labels_species = []
    variables = []

    for idx, row in df.iterrows():
        # Load and save the image as an array
        # img = load_img(row["image_path"], target_size=image_size)
        img = preprocess_image(row["image_path"], image_size)
        img_arr = img_to_array(img)
        images.append(img_arr)

        # Append the class to the list of labels
        labels_class.append(row["class_idx"])

        labels_poison.append(row["poisonous"])
        labels_species.append(row["species_idx"])
        variables.append((row["latitude"], row["longitude"], row["elevation"], row["countryCode"], row["region"], row["substrate"], row["habitat"], row["landcover"]))

    # Stack and convert into a numpy array
    images = np.stack(images)

    # Rescale all of the images so they're pixel value 0 - 1
    images = images / 255.0

    # Cast label list to np.array for easier manipulation
    labels_class = np.array(labels_class)
    labels_poison = np.array(labels_poison)
    labels_species = np.array(labels_species)
    variables = np.array(variables)

    return images, labels_class, labels_poison, labels_species, variables

# ImageFile.LOAD_TRUNCATED_IMAGES = True

# def load_images_and_labels(df, image_size):
#     """Load the images and labels based on the metadata frame passed in."""
#     images = []
#     labels = []

#     for idx, row in df.iterrows():
#         try:
#             # Load and save the image as an array
#             img = load_img(row["image_path"], target_size=image_size)
#             img_arr = img_to_array(img)
#             images.append(img_arr)

#             # Append the class to the list of labels
#             labels.append(row["class"])
#         except (OSError, FileNotFoundError) as e:
#             # Handle corrupted or missing image files
#             print(f"Skipping image {row['image_path']} due to error: {e}")

#     # Stack and convert into a numpy array
#     images = np.stack(images)

#     # Rescale all of the images so they're pixel value 0 - 1
#     images = images / 255.0

#     # Cast label list to np.array for easier manipulation
#     labels = np.array(labels)

#     return images, labels

In [7]:
# Set Variables
IMAGE_SIZE = (224, 224)  # The size images should be rescaled to. If None, defaults to original size
MIN_SAMPLES = 5  # The minimum number of samples needed to be included in a prediction

# Load the metadata
md_df = load_metadata()
    
# Filter out all the fungi that don't have the min number of samples
# This might have been dropping the full test set oops
# md_df = filter_low_counts(md_df, MIN_SAMPLES)

# Map the class to an ID
le = LabelEncoder()
le.fit(md_df["class"])
md_df["class_label"] = md_df["class"]
md_df["class_idx"] = le.transform(md_df["class"])
le.fit(md_df["species"])
md_df["species_label"] = md_df["species"]
md_df["species_idx"] = le.transform(md_df["species"])

# Load all of the images and labels from the metadata
# This function currently resizes and rescales the images
images, labels_class, labels_poison, labels_species, variables = load_images_and_labels(md_df, 224)



In [15]:
# Re-split the images and their labels
train_idx = md_df["split"] == "train"
val_idx = md_df["split"] == "val"
test_idx = md_df["split"] == "test"

train_images = images[train_idx]
train_labels_class = labels_class[train_idx]
train_labels_poison = labels_poison[train_idx]
train_labels_species = labels_species[train_idx]
train_variables = variables[train_idx]

val_images = images[val_idx]
val_labels_class = labels_class[val_idx]
val_labels_poison = labels_poison[val_idx]
val_labels_species = labels_species[val_idx]
val_variables = variables[val_idx]

test_images = images[test_idx]
test_labels_class = labels_class[test_idx]
test_labels_poison = labels_poison[test_idx]
test_labels_species = labels_species[test_idx]
test_variables = variables[test_idx]


In [16]:
print(f"Shape train images: {train_images.shape}")
print(f"Shape train classes: {train_labels_class.shape}")
print(f"Shape train poison: {train_labels_poison.shape}")
print(f"Shape train species: {train_labels_species.shape}")
print(f"Shape train variables: {train_variables.shape}")
print(f"Shape val images: {val_images.shape}")
print(f"Shape test images: {test_images.shape}")

Shape train images: (7819, 224, 224, 3)
Shape train classes: (7819,)
Shape train poison: (7819,)
Shape train species: (7819,)
Shape train variables: (7819, 8)
Shape val images: (2285, 224, 224, 3)
Shape test images: (1911, 224, 224, 3)


In [17]:
# Shuffle the training images
indices = list(range(train_images.shape[0]))  # create a list of indices of the size of the dataset

shuffled_indices = np.random.permutation(indices)  # shuffle the indices

train_images_shuffled = train_images[shuffled_indices]  # shuffle the rows of the dataset
train_labels_class_shuffled = train_labels_class[shuffled_indices]
train_labels_poison_shuffled = train_labels_poison[shuffled_indices]
train_labels_species_shuffled = train_labels_species[shuffled_indices]
train_variables_shuffled = train_variables[shuffled_indices]


# Shuffle the validation images
indices = list(range(val_images.shape[0]))  # create a list of indices of the size of the dataset
shuffled_indices = np.random.permutation(indices)  # shuffle the indices
val_images_shuffled = val_images[shuffled_indices]  # shuffle the rows of the dataset
val_labels_class_shuffled = val_labels_class[shuffled_indices]
val_labels_poison_shuffled = val_labels_poison[shuffled_indices]
val_labels_species_shuffled = val_labels_species[shuffled_indices]
val_variables_shuffled = val_variables[shuffled_indices]


# Shuffle the test images
indices = list(range(test_images.shape[0]))  # create a list of indices of the size of the dataset
shuffled_indices = np.random.permutation(indices)  # shuffle the indices
test_images_shuffled = test_images[shuffled_indices]  # shuffle the rows of the dataset
test_labels_class_shuffled = test_labels_class[shuffled_indices]
test_labels_poison_shuffled = test_labels_poison[shuffled_indices]
test_labels_species_shuffled = test_labels_species[shuffled_indices]
test_variables_shuffled = test_variables[shuffled_indices]

In [18]:
# Add some data augmentation!
# Some horizontal flips? Random crops?

def data_preprocessing(X, labels_class, labels_poison, labels_species, labels_variables, data_partition='train'):
    '''Apply transformations and augmentations to training, validation, and test data;'''

    CONTRAST_FACTOR = 3
    DELTA = 0.3
    
    # image augmentation on training data
    if data_partition=="train":
        # adjust brightness
        X_augm = tf.image.adjust_brightness(X, delta=DELTA) # FILL IN CODE HERE #

        # adjust contrast
        X_augm = tf.image.adjust_contrast(X_augm, contrast_factor=CONTRAST_FACTOR) # FILL IN CODE HERE #

        # random flip
        X_augm = tf.image.flip_left_right(X_augm) # FILL IN CODE HERE #

        # concatenate original X and augmented X_aug data
        X = tf.concat([X, X_augm],axis=0) # FILL IN CODE HERE #

        # concatenate y_train (note the label is preserved)
        labels_class_augm = labels_class
        labels_class = tf.concat([labels_class, labels_class_augm],axis=0)

        labels_poison_augm = labels_poison
        labels_poison = tf.concat([labels_poison, labels_poison_augm],axis=0)

        labels_species_augm = labels_species
        labels_species = tf.concat([labels_species, labels_species_augm],axis=0)

        labels_variables_augm = labels_variables
        labels_variables = tf.concat([labels_variables, labels_variables_augm],axis=0)

        # shuffle X and y, i.e., shuffle two tensors in the same order
        shuffle = tf.random.shuffle(tf.range(tf.shape(X)[0], dtype=tf.int32))
        X = tf.gather(X, shuffle).numpy() # transform X back to numpy array instead of tensor
        labels_class = tf.gather(labels_class, shuffle).numpy() # transform y back to numpy array instead of tensor
        labels_poison = tf.gather(labels_poison, shuffle).numpy()
        labels_species = tf.gather(labels_species, shuffle).numpy()
        labels_variables = tf.gather(labels_variables, shuffle).numpy()
        
        
    # rescale image by dividing each pixel by 255.0 
    # FILL IN CODE HERE #
    X = X / 255.0
    
    return X, labels_class, labels_poison, labels_species, labels_variables

In [19]:
# apply data preprocessing
train_images_shuffled, train_labels_class_shuffled, train_labels_poison_shuffled, train_labels_species_shuffled, train_variables_shuffled = data_preprocessing(train_images_shuffled, train_labels_class_shuffled, train_labels_poison_shuffled, train_labels_species_shuffled, train_variables_shuffled, data_partition='train')
val_images_shuffled, val_labels_class_shuffled, val_labels_poison_shuffled, val_labels_species_shuffled, val_variables_shuffled = data_preprocessing(val_images_shuffled, val_labels_class_shuffled, val_labels_poison_shuffled, val_labels_species_shuffled, val_variables_shuffled, data_partition='val')
test_images_shuffled, test_labels_class_shuffled, test_labels_poison_shuffled, test_labels_species_shuffled, test_variables_shuffled = data_preprocessing(test_images_shuffled, test_labels_class_shuffled, test_labels_poison_shuffled, test_labels_species_shuffled, test_variables_shuffled, data_partition='test')

# print shapes
print('Shape of train images ', train_images_shuffled.shape)
print('Shape of train labels ', train_labels_class_shuffled.shape)
print('Shape of train labels ', train_variables_shuffled.shape)
print('Shape of val images ', val_images_shuffled.shape)
print('Shape of test images ', test_images_shuffled.shape)

Shape of train images  (15638, 224, 224, 3)
Shape of train labels  (15638,)
Shape of train labels  (15638, 8)
Shape of val images  (2285, 224, 224, 3)
Shape of test images  (1911, 224, 224, 3)


In [ ]:
# pd.Series(train_labels_class).value_counts()  # 3156
# pd.Series(train_labels_class).sum()  # 89628
3156 / 89628  # 0.0352

0.035212210469942426

In [ ]:
# fig, axs = plt.subplots(33,1)
# # fig.subplots_adjust(hspace = 0.5, wspace= 0.5)

# for i, label in enumerate(np.unique(train_labels_class)):  # iterate through the unique labels
#     ax = axs[i]
#     image = array_to_img(train_images[train_labels_class == label][0])  # get the first image of the current label
#     ax.imshow(image)  # plot the image
#     ax.set_title(label)
#     ax.axis('off')

# plt.show()

In [21]:
# Set a random seed and clear back end
tf.keras.backend.clear_session()
tf.random.set_seed(1234)

# Convolutional Layer
conv_layer = tf.keras.layers.Conv2D(32, kernel_size=4, padding="same", activation="relu")

# Pooling Layer
pooling_layer = tf.keras.layers.MaxPool2D()

# Dropout Layer
dropout_layer = tf.keras.layers.Dropout(0.25)

# Flattening
flat_layer = tf.keras.layers.Flatten()

# Dense (Multiclassification Layer)
num_classes = len(set(train_labels_class_shuffled))
softmax_layer = tf.keras.layers.Dense(num_classes)

In [22]:
model = tf.keras.Sequential([
    conv_layer,
    pooling_layer,
    dropout_layer,
    flat_layer,
    softmax_layer
])

In [23]:
train_images_shuffled.shape

(15638, 224, 224, 3)

In [24]:
model.build(input_shape=(None, 224, 224, 3))

In [25]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [26]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │         1,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 401408)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 33)             │    13,246,497 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,248,065 (50.54 MB)

 Trainable params: 13,248,065 (50.54 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy',
    verbose=1,
    patience=5,
    mode='max',
    restore_best_weights=True
)

In [28]:
history = model.fit(train_images_shuffled, train_labels_class_shuffled, epochs=10, validation_data=(val_images_shuffled, val_labels_class_shuffled), callbacks=[early_stopping])

Epoch 1/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 103s 207ms/step - accuracy: 0.0350 - loss: 3.8928 - val_accuracy: 0.0000e+00 - val_loss: 3.4965
Epoch 2/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 101s 205ms/step - accuracy: 8.8479e-04 - loss: 3.4965 - val_accuracy: 0.0000e+00 - val_loss: 3.4965
Epoch 3/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 105s 215ms/step - accuracy: 8.8479e-04 - loss: 3.4965 - val_accuracy: 0.0000e+00 - val_loss: 3.4965
Epoch 4/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 110s 225ms/step - accuracy: 8.8479e-04 - loss: 3.4965 - val_accuracy: 0.0000e+00 - val_loss: 3.4965
Epoch 5/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 111s 227ms/step - accuracy: 8.8479e-04 - loss: 3.4965 - val_accuracy: 0.0000e+00 - val_loss: 3.4965
Epoch 6/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 115s 235ms/step - accuracy: 8.8479e-04 - loss: 3.4965 - val_accuracy: 0.0000e+00 - val_loss: 3.4965
Epoch 6: early stopping
Restoring model weights from the end of the best epoch: 1.


In [29]:
model.evaluate(test_images_shuffled, test_labels_class_shuffled)

60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.0000e+00 - loss: 3.4965


[3.4965062141418457, 0.0]